In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-12-28 14:07:17--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   158MB/s    in 0.2s    

2022-12-28 14:07:17 (158 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# combine and clean data
df_combinedBookRating = pd.merge(
    df_ratings,
    df_books,
    on = 'isbn'
    )
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

df_combinedBookRating = df_combinedBookRating.drop(['author'], axis = 1)
df_combinedBookRating.drop_duplicates(subset = ['user', 'title'], keep = 'last', inplace = True)

df_combinedBookRating

,user,isbn,rating,title
0,276725,034545104X,0.0,Flesh Tones: A Novel
1,2313,034545104X,5.0,Flesh Tones: A Novel
2,6543,034545104X,0.0,Flesh Tones: A Novel
3,8680,034545104X,5.0,Flesh Tones: A Novel
4,10314,034545104X,9.0,Flesh Tones: A Novel
...,...,...,...,...
1031170,276688,0517145553,0.0,Mostly Harmless
1031171,276688,1575660792,7.0,Gray Matter
1031172,276690,0590907301,0.0,Triplet Trouble and the Class Trip (Triplet Tr...
1031173,276704,0679752714,0.0,A Desert of Pure Feeling (Vintage Contemporaries)


In [ ]:
# remove from the dataset users with less than 200 ratings and books with less than 100 ratings
users = df_ratings.groupby(['user']).count()
books = df_ratings.groupby(['isbn']).count()

userToRemove = users[users['rating'] < 200]
bookToRemove = books[books['rating'] < 100]

df_bookRating = df_combinedBookRating[df_combinedBookRating.user.isin(list(userToRemove.index)) == False]
df_bookRating = df_bookRating[df_bookRating.isbn.isin(list(bookToRemove.index)) == False]

df_bookRating

,user,isbn,rating,title
65,3363,0446520802,0.0,The Notebook
74,23768,0446520802,6.0,The Notebook
77,27617,0446520802,9.0,The Notebook
78,28204,0446520802,0.0,The Notebook
79,29855,0446520802,0.0,The Notebook
...,...,...,...,...
521237,252071,0312971125,0.0,Crazy for You
521239,258185,0312971125,0.0,Crazy for You
521240,263877,0312971125,0.0,Crazy for You
521241,265115,0312971125,10.0,Crazy for You


In [ ]:
# convert data to 2D matrix
pivot_bookRating = df_bookRating.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
matrix_bookRating = csr_matrix(pivot_bookRating)

pivot_bookRating

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,6543,6563,6575,7158,7286,7346,7915,8067,8245,8681,8936,9856,10447,10819,11601,11676,11993,12538,12824,12982,13082,13273,13552,13850,14422,14521,15408,15418,15957,16106,16634,16795,16916,16966,17950,18067,18401,19085,19664,20115,20201,21014,21252,21576,21659,22625,23288,23768,23872,23902,24194,24921,25409,25601,25981,26516,26535,26544,26583,26593,26883,27617,27647,28204,28360,28523,28591,28634,29259,29855,30276,30487,30509,30511,30533,30711,30735,30972,31315,31391,31556,31826,31846,32195,32440,32721,32773,33974,35050,35433,35836,35857,35859,36554,36606,36609,36836,36907,37567,37712,37950,38023,38273,38556,38781,39281,39467,39616,39646,39773,40889,40943,41084,41841,42400,42914,43246,43806,43842,44296,44595,44728,46398,47316,48494,49109,49460,49889,50225,50547,50730,51094,51350,51386,51450,51883,52199,52203,52584,52614,52853,52917,53174,53628,53729,54218,55187,55490,55492,55548,55734,55892,56271,56360,56399,56447,56856,56959,59172,59727,59971,60244,60277,60337,60707,61619,62272,62891,62895,63394,63714,63938,64436,65258,66680,67840,68555,69042,69078,69232,69355,69405,69697,69808,69971,70052,70415,70594,71712,72992,73394,73651,73681,75591,75860,76151,76223,76352,76499,76626,76818,76942,77809,77940,78553,78783,78834,78973,79186,79441,80538,80683,81045,81492,81560,81977,82407,82511,82831,82893,82926,83287,83637,83671,84024,85426,85526,85656,85701,85757,85993,86202,86243,86641,87141,87143,87555,87746,87974,88283,88677,88693,88733,88793,88937,89192,89602,91203,91342,91832,91931,92547,92652,92810,92853,92979,93047,93085,93363,93629,94242,94347,94853,94923,94985,95010,95193,95316,95359,95903,95932,95991,96054,96448,97874,98297,98391,98741,98758,99955,100459,100644,100846,100906,101209,101305,101851,101876,102275,102359,102647,102702,102967,104399,104429,104636,104665,105374,105517,105979,106225,106816,107021,107301,107453,107784,107951,108005,108285,108352,109461,109574,109901,109955,110029,110214,110483,110912,110934,110973,111174,111947,112001,112026,112881,113270,113519,113817,113904,113983,114178,114368,114414,114444,114865,114868,114988,115002,115003,115120,115490,116122,116599,117251,117539,119575,120093,120548,121726,122429,122793,122881,123625,123790,123883,123981,124078,124079,124363,124487,124942,125039,125203,125519,125692,125774,126492,126604,126736,127200,127233,127359,127429,127914,128208,128696,128835,129008,129074,129358,129465,129716,129851,130474,130554,130571,131027,131046,131402,131675,131837,131855,132083,133571,133689,133747,134837,135045,135149,135265,135458,135831,136010,136139,136205,136252,136382,137190,137589,137688,138073,138097,138441,138578,138844,139467,139742,140000,140036,140358,141493,141710,141819,141901,141902,142093,142454,142524,143175,143253,143415,143792,144131,144531,144555,145165,145449,145451,145619,145641,146113,146230,146348,147141,147451,147847,147965,148199,148258,148744,149069,149907,149908,149934,150124,150498,150968,150979,151589,151790,151806,152186,152249,153563,153621,153662,153718,154176,154992,155014,155147,155219,155495,155916,156150,156269,156467,157247,157273,157811,158295,159033,159376,159858,160032,160541,160819,161752,162052,162639,162738,163804,163973,164027,164096,164323,164465,164533,164675,164828,165232,165308,165319,166123,166596,167349,167471,167800,168047,168064,168144,168245,168387,168464,169233,169663,169699,170184,170229,170513,170518,170575,170634,170742,170947,171118,172030,172512,172742,172888,173291,173415,173632,173684,173835,174092,174216,174304,174791,175003,175886,177072,177090,177374,177432,177458,177690,178181,178199,178667,178950,179733,179744,179772,179978,180348,180586,180651,180917,180957,181176,181687,182085,182086,182987,182993,183196,183995,184299,185233,185308,185384,186570,187145,187170,187256,187517,187520,188010,188951,189334,189516,189835,189973,190459,190708,190807,190925,191187,192093,193499,194600,194669,195694,196047,196077,196160,196457,196502,197364,197659,197775,198621,198699,198711,199

In [ ]:
# create KNN model
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(matrix_bookRating)


NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book, []]
  index = pivot_bookRating.loc[book]

  distances, indices = model.kneighbors([index.values], n_neighbors = 6)

  for i in range(len(distances.flatten()) - 1, 0 , -1):
    recommended_books[1].append([pivot_bookRating.index[indices.flatten()[i]], distances.flatten()[i]])

  return recommended_books


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
